# CNN - Project

## Student Identity

In [ ]:
# @title #### Student Identity
student_id = "" # @param {type:"string"}
name = "Nur Ikhsan Wibowo" # @param {type:"string"}
drive_link = ""  # @param {type:"string"}

assignment_id = "00_cnn_project"

## Import Package

In [ ]:
%pip install -q rggrader
from rggrader import submit, submit_image

Note: you may need to restart the kernel to use updated packages.


## Project Description

In this CNN Project, you will create your own custom Image Classification. You can collect a dataset of images you are interested in and train a CNN model to differentiate between them. For example, a model could be trained to distinguish between different types of birds, cars, plants, or any other topic of interest.

In [ ]:
# %pip install -q torch
# %pip install -q torchvision

In [ ]:
# %pip install opencv-python

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
print(torch.cuda_version)

12.1


In [ ]:
# %pip install -q scikit-learn

In [ ]:
# %pip install matplotlib plotly plotly-express

In [ ]:
# %pip install nbformat

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Task-1 Load the dataset

In this task, you will prepare and load your dataset. **You can choose any dataset you want**, make sure the data is diverse and large enough to prevent overfitting and improve the model's ability to generalize.

If you are using images from the internet, **please respect copyright and privacy laws**. Creative Commons licenses or public domain images are a safe bet, and many APIs (like the Unsplash API) provide access to a large number of such images.

### 1.1 Optional Custom Dataset
Provided below is a custom dataset template that you may want to use for your code. It's completely optional.

Alternatively, you can review the material on Data Augmentation or read the Pytorch tutorial https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import glob
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pathlib

In [ ]:

# # Load CSV file
# csv_file_path = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/Training_set.csv"  # Update with the actual path to CSV file
# df = pd.read_csv(csv_file_path)

# # Path to the folder containing images
# images_folder = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/train"  # Update with the actual path to images folder

# # Iterate through rows in the DataFrame
# for index, row in df.iterrows():
#     file_name = row['filename']
#     label = row['label']

#     # Create a folder for the label if it doesn't exist
#     label_folder = os.path.join(images_folder, str(label))
#     os.makedirs(label_folder, exist_ok=True)

#     # Move the image to the corresponding label folder
#     old_image_path = os.path.join(images_folder, file_name)
#     new_image_path = os.path.join(label_folder, file_name)

#     # Move the image to the label folder
#     shutil.move(old_image_path, new_image_path)

# print("Images organized into folders based on labels.")


In [ ]:
# from sklearn.model_selection import train_test_split
# import shutil

# # Path to the root folder containing label folders
# root_folder = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/train"  # Update with the actual path to your root folder

# # Path to the folder where you want to create the training and testing folders
# output_folder = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/train"  # Update with the desired path

# # Create the training and testing folders if they don't exist
# train_folder = os.path.join(output_folder, 'training')
# test_folder = os.path.join(output_folder, 'testing')

# os.makedirs(train_folder, exist_ok=True)
# os.makedirs(test_folder, exist_ok=True)

# # Iterate through label folders
# for label_folder in os.listdir(root_folder):
#     label_folder_path = os.path.join(root_folder, label_folder)

#     # List all images in the label folder
#     images = [f for f in os.listdir(label_folder_path) if f.endswith('.jpg')]

#     # Check if there are enough images for splitting
#     if len(images) < 2:
#         print(f"Skipping label {label_folder} due to insufficient images for splitting.")
#         continue

#     # Split the images into training and testing sets
#     train_images_label, test_images_label = train_test_split(images, test_size=0.2, random_state=42)

#     # Create subfolders for each label in the training and testing folders
#     train_label_folder = os.path.join(train_folder, label_folder)
#     test_label_folder = os.path.join(test_folder, label_folder)

#     os.makedirs(train_label_folder, exist_ok=True)
#     os.makedirs(test_label_folder, exist_ok=True)

#     # Move the images to the respective subfolders
#     for img in train_images_label:
#         src_path = os.path.join(label_folder_path, img)
#         dest_path = os.path.join(train_label_folder, img)
#         shutil.move(src_path, dest_path)

#     for img in test_images_label:
#         src_path = os.path.join(label_folder_path, img)
#         dest_path = os.path.join(test_label_folder, img)
#         shutil.move(src_path, dest_path)

# print("Images moved to separate training and testing folders.")


### 1.2 Write your code in the block below

In the code block below, prepare and load your dataset. Please include data preprocessing steps such as dividing the dataset into training, validation, and test sets, or data augmentation techniques that you used if any in this section. Do not put the code to build your model here.

Some techniques you may use:
- Find and load existing dataset from Huggingface or Kaggle. (Easy)
- Create your own custom dataset from the images you have in your possesion or internet search and load the dataset. (Hard)
- Etc.

Hint:
- Usually the dataset are loaded into train_dataset and test_dataset

In [ ]:
# Example data transformations
data_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create a custom dataset class to store augmented data
class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, original_dataset, data_augmentation):
        self.original_dataset = original_dataset
        self.data_augmentation = data_augmentation

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, label = self.original_dataset[idx]
        augmented_image = self.data_augmentation(image)
        return augmented_image, label

In [ ]:
# path for training and testing folder
train_path = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/training"
test_path = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/testing"

# transform image to tensor
transform = transforms.Compose([transforms.ToTensor()])

# create dataset
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [ ]:
# apply transformation and augmentation
aug_train = AugmentedDataset(ImageFolder(train_path), data_augmentation)
aug_test = AugmentedDataset(ImageFolder(test_path), data_augmentation)

In [ ]:
from torch.utils.data import ConcatDataset

# combine the training dataset
num_images = len(train_dataset)
print("Number of images in the dataset (before):", num_images)

# Combine the original and augmented datasets
combined_train = ConcatDataset([train_dataset, aug_train])

# Assuming you have a dataset object, e.g., mnist_train
num_images = len(combined_train)
print("Number of images in the dataset (after):", num_images)


Number of images in the dataset (before): 5167
Number of images in the dataset (after): 10334


In [ ]:
# combine the testing dataset
num_images = len(test_dataset)
print("Number of images in the dataset (before):", num_images)

# Combine the original and augmented datasets
combined_test = ConcatDataset([test_dataset, aug_test])

# Assuming you have a dataset object, e.g., mnist_train
num_images = len(combined_test)
print("Number of images in the dataset (after):", num_images)

Number of images in the dataset (before): 1332
Number of images in the dataset (after): 2664


In [ ]:
# Create DataLoader
batch_size = 32

train_loader = DataLoader(combined_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(combined_test, batch_size=batch_size, shuffle=True)


In [ ]:
# define categories from butterfly image classification
root = pathlib.Path(train_path)
categories = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(categories)

['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO', 'ATALA', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BECKERS WHITE', 'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROWN SIPROETA', 'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHECQUERED SKIPPER', 'CHESTNUT', 'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMMON BANDED AWL', 'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH', 'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE', 'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'GOLD BANDED', 'GREAT EGGFLY', 'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREY HAIRSTREAK', 'INDRA SWALLOW', 'IPHICLUS SISTER', 'JULIA', 'LARGE MARBLE', 'MALACHITE', 'MANGROVE SKIPPER', 'MESTRA', 'METALMARK', 'MILBERTS TORTOISESHELL', 'MONARCH', 'MOURNING CLOAK', 'ORANGE OAKLEAF', 'ORANGE TIP', 'ORCHARD SWALLOW', 'PAINTED LADY', 'PAPER KITE', 'PEACOCK', 'PINE WHITE', 'PIPEVINE SWALLOW', 'POPINJAY', 'PURPLE HAIRSTREAK', 'PURPLISH COPPER', 'QUESTION MARK', 'RED ADMIRA

In [ ]:
len(categories)

75

### 1.3 Submission

You'll submit the link to your dataset for Task-1.

- If you use existing dataset from Kaggle or Huggingface, then you can put the link to those dataset here.

- If you use your own custom dataset, Upload and store the custom dataset in your Google Drive that you shared with us and put the link to the folder containing that dataset here.

In [ ]:
# Submit Method
dataset_link = "https://drive.google.com/drive/folders/1Ji_4eCycBJ_5qj5kQ2pgVd73-c2B7udf?usp=drive_link" # Put your model link

question_id = "00_cnn_project_dataset"
submit(student_id, name, assignment_id, dataset_link, question_id, drive_link)

'Assignment successfully submitted'

## Task-2 Build your model

In this task, you will now build and save your model. You can either create your own CNN model or choose any pretrained model that you feel is most appropriate for your dataset.

### 2.1 Write your code in the block below

In the code block below, write the code to **create your model, either from scratch or fine tuning a pretrained model**. You will need to write the code for your model definition, such as the layers used, loss function and optimizer. Please include also the training and validation loops.

Make sure you **save your model to a file** and **measure the accuracy of your model**, as this will be submitted for this task.

Some techniques you may use:
- Use pretrained model. (Easy)
- Create a CNN model from scratch. (Hard)
- Etc.

Hint:
- Use GPU in Google Colab, it significantly improves the time taken for training, compared to CPU.
- **Google Colab GPU usage for free-tier have a limit**, which is unknown, so I suggest you try out in CPU mode that your code works without error, then use GPU for traininig.
- If you are going to upload to Huggingface by using the Transformer Trainer during training, make sure you use the Huggingface method. Refer to Transfer Learning section or read the documentation here: https://huggingface.co/docs/transformers/model_sharing

In [ ]:
import torch.optim as optim

In [ ]:
# load the pretrained model
model = models.resnet18(pretrained=True)

c:\Users\SoulPerformPC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SoulPerformPC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# # freeze all layers to use Transfer learning method
# for param in model.parameters():
#     param.requires_grad = False

# unfreeze last layer (fully connected layer)
for param in model.fc.parameters():
    param.requires_grad = True

# fine-tuning last layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 75) # 75 categories for butterfly image dataset


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#Write your code to build your model here

best_accuracy = 0
num_epochs = 30

# Train the model
for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        # Calculate the loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Update the weights
        optimizer.step()

        # calculate train_loss and train_accuracy
        train_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data, 1)

        train_accuracy += int(torch.sum(prediction==labels.data))

    train_accuracy = train_accuracy/ len(combined_train)
    train_loss = train_loss / len(combined_train)

# Test the model
    model.eval()
    test_accuracy = 0.0

    for i, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction==labels.data))

    test_accuracy = test_accuracy / len(combined_test)

    # Print the accuracy
    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Accuracy: {:.2f}".format(epoch, train_loss, train_accuracy, test_accuracy))

    # Save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), "butterfly_resnet_checkpoint.model")
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: 3.00 Train Accuracy: 0.37 Test Accuracy: 0.70
Epoch: 1 Train Loss: 1.18 Train Accuracy: 0.76 Test Accuracy: 0.78
Epoch: 2 Train Loss: 0.75 Train Accuracy: 0.83 Test Accuracy: 0.83
Epoch: 3 Train Loss: 0.60 Train Accuracy: 0.86 Test Accuracy: 0.85
Epoch: 4 Train Loss: 0.51 Train Accuracy: 0.88 Test Accuracy: 0.85
Epoch: 5 Train Loss: 0.43 Train Accuracy: 0.90 Test Accuracy: 0.86
Epoch: 6 Train Loss: 0.39 Train Accuracy: 0.90 Test Accuracy: 0.87
Epoch: 7 Train Loss: 0.35 Train Accuracy: 0.91 Test Accuracy: 0.87
Epoch: 8 Train Loss: 0.32 Train Accuracy: 0.92 Test Accuracy: 0.88
Epoch: 9 Train Loss: 0.32 Train Accuracy: 0.92 Test Accuracy: 0.89
Epoch: 10 Train Loss: 0.28 Train Accuracy: 0.93 Test Accuracy: 0.89
Epoch: 11 Train Loss: 0.28 Train Accuracy: 0.93 Test Accuracy: 0.88
Epoch: 12 Train Loss: 0.26 Train Accuracy: 0.94 Test Accuracy: 0.89
Epoch: 13 Train Loss: 0.25 Train Accuracy: 0.94 Test Accuracy: 0.89
Epoch: 14 Train Loss: 0.23 Train Accuracy: 0.94 Test Accur

In [ ]:
accuracy = 0.90 #if you change the name of your accuracy variable, change the name also in the submit parameter

### 2.2 Submission

You'll submit the link to your model and the accuracy score for your model in this section.

Remember to download or move/upload your model from Google Colab to your Google Drive. Put the link to the Google Drive for your model below.

If you use a different variable to store your accuracy, remember to also change the `accuracy` variable below.

In [ ]:
# Submit Method
model_link = "https://drive.google.com/file/d/1DP50zKxAZFV5QxnITX85lhHNhRVj0WJc/view?usp=drive_link" # Put your model link

question_id = "01_cnn_project_model_link"
submit(student_id, name, assignment_id, model_link, question_id, drive_link)
question_id = "02_cnn_project_model_accuracy"
submit(student_id, name, assignment_id, str(accuracy), question_id, drive_link)

'Assignment successfully submitted'

## Task-3 Model Inference

In this task, you will be exercising the application of your model, or as it's commonly referred to in AI terminology, you will be performing inference using your model.

Simply load your saved model from Task-2 and create an inference for the model. Where you'll feed an image as input and the model will output the label as well as the percentage of confidence for the label.

### 3.1 Write your code in the block below

In the code block below write the code to use the model you created in Task-2. Load the model and input image, afterwards, show the result of the label/class together with confidence level in percentage as well as the input image.

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import time
import torch.nn.functional as F
from PIL import Image
import cv2
import numpy as np

In [ ]:
# load the pretrained model
model_for_predict = models.resnet18(pretrained=True)

# unfreeze last layer (fully connected layer)
for param in model_for_predict.fc.parameters():
    param.requires_grad = True

# fine-tuning last layer
num_ftrs = model_for_predict.fc.in_features
model_for_predict.fc = nn.Linear(num_ftrs, 75) # 75 categories for butterfly image dataset

# load the saved model
checkpoint = torch.load('butterfly_resnet_checkpoint.model')
model_for_predict.load_state_dict(checkpoint)

# predict the images
model_for_predict.eval()

c:\Users\SoulPerformPC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\SoulPerformPC\anaconda3\envs\pt\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#Write your code for inference here

# folder path containing unlabelled butterfly images
pred_path = "C:/Users/SoulPerformPC/Documents/Ikhsan_Latihan/project_bootcamp/butterfly classification images/predict"

# Get all image files in the folder
image_files = [f for f in os.listdir(pred_path) if f.lower().endswith(('.jpg'))]

# Create a list of image paths
image_paths = [os.path.join(pred_path, img) for img in image_files]

In [ ]:
# Define the transforms
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Set the delay time between images (in seconds)
delay_time = 3

# Loop through the images
for i in range(len(image_paths)):
    img_path = os.path.join(pred_path, image_paths[i])

    # Load and transform the image
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Perform prediction
    with torch.no_grad():
        output = model_for_predict.to(device)(img_tensor.to(device))

    # Get predicted class and confidence
    predicted_class = torch.argmax(output).item()
    confidence = torch.nn.functional.softmax(output, dim=1)[0][predicted_class].item()

    # Convert class index to class name
    class_name = categories[predicted_class]

    # Display the image with predicted class and confidence
    img_display = px.imshow(img)
    img_display.update_layout(title_text=f"Class: {class_name}, Confidence: {confidence * 100:.2f}%", title_x=0.5)
    img_display.show()

    # Introduce a delay between images
    time.sleep(delay_time)

    # Clear the output for the next iteration
    torch.cuda.empty_cache()  # Clear GPU memory (if using GPU)
    clear_output(wait=True)


### 3.2 Submission

You'll submit a screenshot of your inference in this section. Remember to save the screenshot first before submitting it.

Hint:

![Upload colab](https://storage.googleapis.com/rg-ai-bootcamp/project-3-pipeline-and-gradio/upload-colab.png)

- In Google Colab you can just use the "Folder" sidebar and click the upload button. Make sure your screenshot match below requirements:

    - Image name screenshot is `submission.jpg`. If you change the name of the screenshot  file, change it also in the submit_image parameter.
    - The input image and label as well as percentage of confidence should be included in the screenshot

Here is an example of a correct screenshot:

![Screenshot submission sammple - hummer](https://storage.googleapis.com/rg-ai-bootcamp/projects/project-5-cnn-hummer.png)

In [ ]:
# Submit Method

question_id = "03_cnn_project_inference"
submit_image(student_id, question_id, './submission.jpg')

'Assignment successfully submitted'

## Task-4 Model Publishing

In this task, you will push your model to Huggingface. Once you've pushed your model to the Hugging Face Model Hub, you'll have a link that points directly to your model's page. You can share this link with others, and they can use it to directly load your model for their own uses.

### 4.1 Write your code in the block below

In the code block below, write the code to push your model to Huggingface. There are several methods to do this, please refer to the documentation: https://huggingface.co/docs/transformers/model_sharing

Some techniques you may use:
- If you use the Transformer Trainer during the training loop when you create your model above, then you can simply put your `trainer.push_to_hub()` here.
- You can also use the web interface on Huggingface.

Hint:
- Remember to login first to your Huggingface account.
- If you are pushing programmaticaly, then use the huggingface-cli to login.

In [ ]:
# #Write your code for publishing here
# %pip install -q transformers
# %pip install -q huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### 4.2 Submission

You'll submit a a link to your huggingface model in this section.

The following link is an example of what a trained model's page looks like: https://huggingface.co/aditira/emotion_classification. This is not your model, but rather an example of what your final result might resemble.

Remember, for this project you should push your output model to your own Hugging Face account. The link for your model will be different and should reflect your own username and model name.

In [ ]:
# Submit Method
huggingface_model_link = "https://huggingface.co/SoulPerforms/Butterfly_image_classification_resnet18" # Put your model link

question_id = "04_cnn_project_publish"
submit(student_id, name, assignment_id, huggingface_model_link, question_id, drive_link)

'Assignment successfully submitted'

FIN